In the session 4 we trained a model for predicting churn and evaluated it. Now let's deploy it:

In [1]:
# Import libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
# Load churn data and perform preprocessing
df = pd.read_csv('../03-classification/churn-data.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [3]:
# Split to train and test
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [4]:
# Lists of categorical and numerical columns
numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [5]:
# Function to train model
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)
    
    return dv, model

In [6]:
# Function to make predictions
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1] # extract second columns of all rows

    return y_pred

In [7]:
# Parameters for KFold cross validation
C = 1.0
n_splits = 5

In [8]:
# Apply KFold cross validation
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = [] # list to store AUC scores

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.churn.values
    y_val = df_val.churn.values

    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print(f'C={C} {np.mean(scores):.3f} {np.std(scores):.3f}')

C=1.0 0.840 0.008


In [9]:
# AUC scores after 5 iterations (n_splits=5)
scores

[0.8420649810426168,
 0.8455854357038802,
 0.8311780052177403,
 0.8301724275756219,
 0.8522677355589721]

In [10]:
# Train model on full train data and make predictions on test data
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)
y_test = df_test.churn.values

# Calculate AUC score on test data
auc = roc_auc_score(y_test, y_pred)
auc

0.8572386167896259

Save the model

In [11]:
import pickle

In [12]:
output_file = f'model_C={C}.bin' # parameter C is C=1.0

In [13]:
output_file

'model_C=1.0.bin'

First method of saving a file (not desirable)

In [14]:
f_out = open(output_file, 'wb')
pickle.dump((dv, model), f_out)
f_out.close() # must close the file manually

In [15]:
# Check the model size and file permissions
!ls -lh *.bin

-rw-r--r-- 1 awon 197121 2.4K Oct  4 01:11 model_C=1.0.bin


Second method of saving file (desirable)

In [16]:
# Save file using context manager
with open(output_file, 'wb') as f_out:
    pickle.dump((dv, model), f_out)

Load the model

In [1]:
import pickle

In [2]:
input_file = 'model_C=1.0.bin'

In [3]:
with open(input_file, 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [4]:
dv, model

(DictVectorizer(sparse=False), LogisticRegression(max_iter=1000))

Test the loaded model on dummy customer:

In [5]:
# Customer data in dictionary format
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [6]:
# Transform customer data
X = dv.transform([customer])

In [7]:
# Make predictions
y_pred = model.predict_proba(X)[0, 1]

In [8]:
print(f'input: {customer}')
print(f'output: {y_pred}')

input: {'gender': 'female', 'seniorcitizen': 0, 'partner': 'yes', 'dependents': 'no', 'phoneservice': 'no', 'multiplelines': 'no_phone_service', 'internetservice': 'dsl', 'onlinesecurity': 'no', 'onlinebackup': 'yes', 'deviceprotection': 'no', 'techsupport': 'no', 'streamingtv': 'no', 'streamingmovies': 'no', 'contract': 'month-to-month', 'paperlessbilling': 'yes', 'paymentmethod': 'electronic_check', 'tenure': 1, 'monthlycharges': 29.85, 'totalcharges': 29.85}
output: 0.6363584152746705


In [26]:
import requests

In [27]:
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [28]:
url = 'http://localhost:9696/predict'

In [29]:
# Send post request to our local host url
response = requests.post(url, json=customer) # send customer dict as json

# Check reponse (200 means the request has succeeded)
response 

<Response [200]>

In [30]:
# Use 'json()' function to see the body of the response
response = response.json()
response

{'churn': True, 'churn_probability': 0.6363584152746705}

In [31]:
# Conditional to send promotional email
if response['churn']:
    print('Sending email to', 'asdx-123d')

Sending email to asdx-123d
